<a href="https://colab.research.google.com/github/martaherraiz/APRENDIZAJE_AUTOMATICO_23_24/blob/main/4_1_Redes_Neuronales_Convolucionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neuronales Convolucionales


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)#

2.6.0


## Datos digits MNIST
la maxpooling es otro tipo de capa, antes usabamos densas.
la capa convolucional recorre en una ventana la imagen de izq a dcha de arriba a abajo aplicándole los valores de una matriz. Unos valores aleatorios, esto lo llamamos mapa de características. Con la matriz nueva transformada le aplico transformaciones convolucionales de la misma forma hasta llegar al resultado, aplico la función softmax y me dice la probabilidad de que sea un 0 o un 1, 2... calculamos con categorical_crossentropy y con la retroalimentacion del gradiente ajusto la matriz continuamente.

cuando uso el gimp (la aplicación) a la imagen le hago una transformación y me devuelve una imagen con una transformacion que a mi me interesa, en redes convolucionales es al revés.
La matriz tiene valores aleatorios que no conozco y al final me da la probabilidad de que sea lo que busco.

Las capas iniciales de la red se quedan con características globales, pero las más cercanas a la respuesta se quedan con detalles como un ojo o una nariz... Aprende que una nariz no puede estar por encima de unos ojos.

Me creo un modelo secuencial y le añado una capa convolucional, kernel = matriz de pesos = filtro. El filtro que es una matriz de pesos vamos a poner 32 filtros. Conv2D(32, ...) ese 32 son los filtros. Estamos dándole la posibilidad de que aprenda 32 características de la imagen. A más filtros más específico. Los 32 están puestos en paralelo. Cada filtro tiene una matriz de pesos diferente, w0, w1... pero los valores de la matriz de pesos son los mismos para los desplazamientos. Los filtros no guardan la imagen en si, cogen características a medida que vamos mirando las profundas, vemos algo negro, luego algo que se parece a un ojo, luego algo que se parece a una cara.

Hay 5x5 = 25 pesos + 1 del sesgo

Matriz x entrada = valor que se pone en el mapa de características.

La ventana deslizante es de 5x5 píxeles Conv2D(32, (5, 5), ...) ese va a ser el mapa de características :

28 - 5 + 1 = 24 -> 24x24 es el nuevo mapa de características, reduce el inicial que es de 28x28. Cojo una imagen y la convierte en un espacio de conocimiento diferente.

La f de activación es la relu, que es la que mejor funciona en estos casos.

La capa de maxpooling coge la capa dela convolucional y la reduce aún más. Si tengo un mapa de características de 24x24, partimos de una imagen de 28x28 que al pasarle la convolución se queda en un mapa de características de 24x24. Luego, al aplicar pooling(2,2) se queda como 12x12.


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
Total params: 832
Trainable params: 832
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
Total params: 52,096
Trainable params: 52,096
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                10250     
Total params: 62,346
Trainable params: 62,346
Non-trainable params: 0
____________________________________________________

In [ ]:
from keras.utils.np_utils import to_categorical

mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print (train_images.shape)
print (train_labels.shape)
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print (train_images.shape)
print (train_labels.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(60000, 28, 28, 1)
(60000, 10)


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(train_images, train_labels,
          batch_size=100,
          epochs=5,
          verbose=1)

test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

Epoch 1/5
600/600 [==============================] - 34s 7ms/step - loss: 0.9671 - accuracy: 0.7378
Epoch 2/5
600/600 [==============================] - 4s 6ms/step - loss: 0.2795 - accuracy: 0.9172
Epoch 3/5
600/600 [==============================] - 4s 6ms/step - loss: 0.1993 - accuracy: 0.9422
Epoch 4/5
600/600 [==============================] - 4s 6ms/step - loss: 0.1588 - accuracy: 0.9534
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.1070 - accuracy: 0.9702
Test accuracy: 0.9702000021934509
